In [ ]:
#for reading text from pdf
!pip install pdfminer.six

In [ ]:
from pdfminer.high_level import extract_text

In [ ]:
# text=extract_text("/content/Lecture Summarizer1.pdf")
# print(text)

In [ ]:
pip install pytesseract

In [ ]:
import pytesseract
from PIL import Image
import os

In [ ]:
# img=Image.open("/content/Image1.png")
# text= pytesseract.image_to_string(img)
# print(text)

## Image/PDF to Text

In [ ]:
#pdf to text
def pdf_to_text(path):
  text=extract_text(path)
  return text

In [ ]:
#image to text
def image_to_text(path):
  img=Image.open(path)
  text=pytesseract.image_to_string(img)
  return text

## Audio/Video to Text

In [ ]:
!pip install moviepy #video to audio library
!pip install -q openai-whisper
!sudo apt update && sudo apt install -y ffmpeg #whisper model for speech to text

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

In [ ]:
import whisper
from moviepy.editor import *

In [ ]:
whisper_model=whisper.load_model("medium")
def audio_to_text(path): #audio to text
  text=whisper_model.transcribe(path)["text"]
  return text

def video_to_text(path):
  video=VideoFileClip(path)
  audio=vc.audio
  p="/content/audio.wav"
  audio.write_audiofile(p)
  video.close()
  text=audio_to_text(p)
  return text

In [ ]:
#Final Function
def to_text(path):
  ext = os.path.splitext(path)[1].lower()
  if ext==".pdf":
    text=pdf_to_text(path)
  elif ext in [".png", ".jpg", ".jpeg"]:
    text=image_to_text(path)
  elif ext in [".mp4", ".avi", ".mov", ".mkv", ".flv"]:
    text=video_to_text(path)
  elif ext in [".mp3", ".wav", ".m4a", ".flac", ".ogg"]:
    text=audio_to_text(path)
  return text

## Generating Chunks

In [ ]:
#Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#initiate text splitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n","\n"," ",""]
)

In [ ]:
def generate_chunks(text):
  chunks=text_splitter.split_text(text) #text to chunks
  return chunks

In [ ]:
# chunks=generate_chunks(text)
# print(chunks[0])

## Creating Documents

In [ ]:
#Creating documents for Vector Database. Necessary for tracking chunks
from langchain.docstore.document import Document

In [ ]:
# docs=[]
# for i,chunk in enumerate(chunks):
#   doc=Document(
#       page_content=chunk,
#       metadata={
#           "page": i+1,
#           "chunk": i
#       }
#   )
#   doc.metadata["source"]=f"{doc.metadata["page"]} - {doc.metadata["chunk"]}"
#   docs.append(doc)
# print(docs[0])

In [ ]:
def generate_document(chunks):
  docs=[]
  for i,chunk in enumerate(chunks):
    doc=Document(
        page_content=chunk,
        metadata={
            "page": i+1,
            "chunk": i
        }
    )
    doc.metadata["source"] = f"{doc.metadata['page']}-{doc.metadata['chunk']}"
    docs.append(doc)
  return docs

## Create Vector Database

In [ ]:
!pip install langchain-community langchain

In [ ]:
!pip -q install sentence_transformers faiss-cpu langchain text-generation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 27.5 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS #Facebook AI Similarity Search
from langchain.embeddings import HuggingFaceEmbeddings #To create embeddings (word to vector)

In [ ]:
hf_embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"
hf_embeddings=HuggingFaceEmbeddings(model_name=hf_embedding_model_name)

  hf_embeddings=HuggingFaceEmbeddings(model_name=hf_embedding_model_name)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#final function to create database
def vector_database(path):
  text=to_text(path)
  chunks=generate_chunks(text)

  docs=generate_document(chunks)
  VectorDB=FAISS.from_documents(docs,hf_embeddings)
  return VectorDB

In [ ]:
# db=vector_database("/content/Lecture Summarizer1.pdf")

#Searching Function

In [ ]:
# result=db.similarity_search("What preprocessing are being used in the project?",k=2)
# context=""
# for i in range(len(result)):
#   context+=result[i].page_content
# print(context)

In [ ]:
def search(query,db):
  result=db.similarity_search(query,k=5)
  context=""
  for i in range(len(result)):
    context+=result[i].page_content
  return context

## ChatBot

In [ ]:
def prompt_creation(query,db):
  context=search(query,db)
  prompt=f""" Act as an analyst and answer the question in a concise manner based on the given context. Strictly Adhere to the given context.
  Question: {query}
  Context: {context}
  Answer:"""
  return prompt

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#downloading falcon
model_name="tiiuae/Falcon3-1B-Instruct"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True).to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.34G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [ ]:
def response(prompt):
  inputs=tokenizer(prompt,return_tensors="pt",padding=True).to(device)
  output=model.generate(**inputs, max_new_tokens=200, num_beams=3)
  result=tokenizer.decode(output[0],skip_special_tokens=True)

  #answer fix: to return only model's response
  if "Answer:" in result:
    result=result.split("Answer:")[-1].strip()
    result=result.split("<|assistant|>")[0].strip()
    result=result.replace("<|assistant|>","").strip()
  return result

In [ ]:
#Final Function
def final(path,query):
  db=vector_database(path)
  prompt=prompt_creation(query,db)
  result=response(prompt)
  return result

In [ ]:
# r=final("/content/Lecture Summarizer1.pdf","What is AI?")

In [ ]:
import gradio as gr

In [ ]:
def QnA(query,db,chat_history): #for Q and A
    prompt=prompt_creation(query,db)
    result=response(prompt)
    chat_history.append((query,result))
    return "",chat_history

#vector_database for creating database
def vector_database(path):
  text=to_text(path)
  chunks=generate_chunks(text)

  docs=generate_document(chunks)
  VectorDB=FAISS.from_documents(docs,hf_embeddings)
  return VectorDB,"DataBase Created Successfully!"

In [ ]:
with gr.Blocks() as demo:

  db_state=gr.State() #an object to hold database
  chatbot=gr.Chatbot(label="QnA Bot") #chatbot

  with gr.Tab("Create Database"):
      file_input = gr.File(label="Upload Knowledge Base")
      create_btn = gr.Button("Create Database")
      db_output = gr.Textbox(label="Status")

      create_btn.click(vector_database, inputs=file_input, outputs=[db_state,db_output])

  with gr.Tab("Q&A Chatbot"):
      question_input = gr.Textbox(label="Ask a Question")
      # ask_btn = gr.Button("Ask")
      question_input.submit(QnA, inputs=[question_input,db_state,chatbot], outputs=[question_input,chatbot])

  clear=gr.ClearButton([question_input,chatbot])

demo.launch(debug=True)

  chatbot=gr.Chatbot(label="QnA Bot") #chatbot



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://22603d01de3ca5bb4c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
